<a href="https://colab.research.google.com/github/leonardolunamoreno/Computadoras-y-programaci-n2021-grupo-1157/blob/main/Afinador_de_guitarra_versi%C3%B3n_de_terminal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

La siguiente es la primera versión del afinador de guitarra.
No cuento con un teléfono celular, de manera que no puedo descargar aplicaciones tales como un afinador de guitarra, así que voy a programar uno.

Versión 1 del afinador de guitarra

In [ ]:
import numpy as np        # Es la librería para números y arreglos. La usamos para la FFT y manipular señales.
import sounddevice as sd  # Controla la grabación/reproducción de audio desde el micrófono.

def detectar_frecuencia():
    duracion = 1.0         # tiempo de grabación en segundos. Con 1 segundo tienes resolución de frecuencia ~1 Hz
    fs = 44100             # frecuencia de muestreo en Hz (44.1 kHz). Debe ser mayor que el doble de la frecuencia máxima que quieras medir (Nyquist). Para guitarra está bien.
                           # ¿Cuáles son todas las frecuencias que debo medir para la guitarra?
                           # ¿Cuales son todas las afinaciones que debo considerar si hago un afinador completo?
                           # Comenzaré considerando la afinación estandar de la guitarra


    print("Toca la cuerda...")
    audio = sd.rec(int(duracion * fs), samplerate=fs, channels=1) # sd.rec(...) empieza a grabar duracion * fs muestras, channels=1 es mono. Devuelve un array de forma (N, 1).
    sd.wait()                                                     # sd.wait() espera hasta que termine la grabación antes de seguir.

    señal = audio.flatten()      # flatten() convierte el array (N,1) en un vector 1-D de longitud N. Ese es el dato que vamos a analizar.
    fft = np.fft.rfft(señal)     # es la FFT real: calcula sólo la mitad positiva del espectro (sirve para  ahorrar trabajo). fft contiene complejos (magnitud + fase).
    freqs = np.fft.rfftfreq(len(señal), 1/fs)   # crea el vector de frecuencias correspondientes a cada bin de la rfft. El k-ésimo elemento corresponde a freqs[k].

    idx = np.argmax(np.abs(fft))   # np.abs(fft) toma la magnitud del espectro. np.argmax(...) devuelve el índice del pico más alto.
    return freqs[idx]              # freqs[idx] es la frecuencia correspondiente a ese pico — el valor que devuelve la función.

while True:                      # Bucle infinito que graba y muestra la frecuencia detectada cada vez.
    f = detectar_frecuencia()
    print(f"Frecuencia detectada: {f:.2f} Hz")


Versión 2 del afinador de guitarra

Para que el siguiente código funcione de forma apropiada, es necesario tener un micrófono activo en el equipo en el cual se probará el siguiente código.

In [ ]:
import numpy as np
import sounddevice as sd
import math

# Notas estándar de guitarra (nota, frecuencia)
guitar_strings = {
    "E2": 82.4069,
    "A2": 110.0000,
    "D3": 146.8324,
    "G3": 196.0000,
    "B3": 246.9417,
    "E4": 329.6276
}

def parabolic_interpolation(mags, peak_idx):
    # Ajuste parabólico para refinar el pico (mejora el estimado de frecuencia)
    if peak_idx <= 0 or peak_idx >= len(mags)-1:
        return peak_idx, mags[peak_idx]
    alpha = mags[peak_idx-1]
    beta  = mags[peak_idx]
    gamma = mags[peak_idx+1]
    p = 0.5 * (alpha - gamma) / (alpha - 2*beta + gamma)
    peak_refined = peak_idx + p
    peak_mag = beta - 0.25*(alpha - gamma)*p
    return peak_refined, peak_mag

def freq_to_note_and_cents(f):
    # Convierte freq a la nota estándar más cercana y devuelve desviación en cents
    # Usaremos la referencia A4 = 440 Hz para cálculo general
    if f <= 0:
        return None, None, None
    # número MIDI aproximado:
    midi = 69 + 12 * math.log2(f / 440.0)
    midi_round = int(round(midi))
    cents = (midi - midi_round) * 100  # diferencia en cents
    # convertimos midi a nota legible
    note_names = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']
    name = note_names[midi_round % 12] + str((midi_round // 12) - 1)
    return name, midi_round, cents

def detectar_frecuencia_mejor(duracion=1.0, fs=44100):
    print("Toca la cuerda...")
    audio = sd.rec(int(duracion * fs), samplerate=fs, channels=1)
    sd.wait()
    señal = audio.flatten().astype(float)

    # Normaliza y aplica ventana
    señal = señal - np.mean(señal)
    N = len(señal)
    ventana = np.hanning(N)
    ventana_señal = señal * ventana

    # Zero-padding para mejorar interpolación de la FFT (factor 4)
    zpad = 4 * N
    fft = np.fft.rfft(ventana_señal, n=zpad)
    mags = np.abs(fft)
    freqs = np.fft.rfftfreq(zpad, 1/fs)

    # Busca el pico (ignoramos DC)
    mags[0:2] = 0
    peak_idx = np.argmax(mags)
    peak_refined, _ = parabolic_interpolation(mags, peak_idx)
    # convertimos el índice refinado a frecuencia
    f_est = freqs[0] + (peak_refined * (freqs[1]-freqs[0]))

    return f_est

if __name__ == "__main__":
    try:
        while True:
            f = detectar_frecuencia_mejor(duracion=1.0, fs=44100)
            if f is None or f <= 0:
                print("No se detectó señal válida.")
                continue
            note, midi, cents = freq_to_note_and_cents(f)
            direction = "bien" if abs(cents) < 5 else ("alta" if cents > 0 else "baja")
            print(f"{f:.2f} Hz → {note} ({cents:+.1f} cents) -> {direction}")
    except KeyboardInterrupt:
        print("\nSalida. Fin.")


Versión 3 del afinador de guitarra

Al igual que en la versión 2 del afinador, es necesario tener un micrófono conectado a la computadora, o en su defecto que la computadora tenga permisos para utilizar el mirófono

In [ ]:
import numpy as np
import sounddevice as sd

import math

# Afinaciones predefinidas
TUNINGS = {
    "E Standard": {
        6: 82.41, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63
    },
    "Drop D": {
        6: 73.42, 5: 110.00, 4: 146.83, 3: 196.00, 2: 246.94, 1: 329.63
    },
    "Eb Standard": {
        6: 77.78, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13
    },
    "Drop C# (Over Now)": {
        6: 69.30, 5: 103.83, 4: 138.59, 3: 185.00, 2: 233.08, 1: 311.13
    }
}

def parabolic_interpolation(mags, peak_idx):
    if peak_idx <= 0 or peak_idx >= len(mags)-1:
        return peak_idx
    alpha = mags[peak_idx-1]
    beta = mags[peak_idx]
    gamma = mags[peak_idx+1]
    p = 0.5 * (alpha - gamma) / (alpha - 2*beta + gamma)
    return peak_idx + p

def detectar_frecuencia(duracion=1.0, fs=44100):
    print("Toca la cuerda...")
    audio = sd.rec(int(duracion * fs), samplerate=fs, channels=1)
    sd.wait()

    señal = audio.flatten().astype(float)
    señal -= np.mean(señal)

    N = len(señal)
    ventana = np.hanning(N)
    señal *= ventana

    zpad = 4 * N
    fft = np.fft.rfft(señal, n=zpad)
    mags = np.abs(fft)
    freqs = np.fft.rfftfreq(zpad, 1/fs)

    mags[:2] = 0
    idx = np.argmax(mags)
    idx_ref = parabolic_interpolation(mags, idx)

    f_est = freqs[0] + idx_ref * (freqs[1]-freqs[0])
    return f_est

def comparar(f_detectada, f_objetivo):
    cents = 1200 * math.log2(f_detectada / f_objetivo)
    if abs(cents) < 5:
        estado = "OK"
    elif cents > 0:
        estado = "ALTA"
    else:
        estado = "BAJA"
    return cents, estado

if __name__ == "__main__":
    print("=== Afinador con Selector de Afinación ===\n")
    print("Elige afinación:")
    for i, t in enumerate(TUNINGS.keys()):
        print(f"{i+1}. {t}")

    choice = int(input("\nSelecciona número: "))
    tuning_name = list(TUNINGS.keys())[choice-1]
    tuning = TUNINGS[tuning_name]

    print(f"\nAfinación seleccionada: {tuning_name}")
    print("Afina cuerda por cuerda: 6 → 1\n")

    try:
        while True:
            cuerda = int(input("¿Qué cuerda estás afinando? (1-6): "))
            objetivo = tuning[cuerda]

            f = detectar_frecuencia()
            print(f"Detectado: {f:.2f} Hz — Objetivo: {objetivo:.2f} Hz")

            cents, estado = comparar(f, objetivo)
            print(f"Diferencia: {cents:+.1f} cents → {estado}\n")

    except KeyboardInterrupt:
        print("\nCerrando afinador.")

# La cuerda 6 es la más gruesa, y la cuerda 1 es la más delgada
